In [16]:
%pip install beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import requests
from bs4 import BeautifulSoup
import time
import random
import json
import os

In [18]:
BASE_URL = "http://www.lily-chou-chou.jp/holic/bbs/htm/index2_mddl.php?ofst="
START_PAGES = 427032 #start from 427032th pages back to 1
LIMIT = 425900 #upto 427000th pages

In [19]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [20]:
# ===== NETWORK =====
CONNECT_TIMEOUT = 5
READ_TIMEOUT = 120     # offset sâu cần read timeout dài

# ===== SPEED CONTROL =====
BASE_SLEEP = 3.0     # tối thiểu 10s / request
JITTER = 5.0          # random thêm 0–5s
MAX_RETRY = 2         # KHÔNG retry nhiều

# ===== BLOCK CONTROL =====
FAIL_THRESHOLD = 3
BLOCK_PAUSE = 5 * 60   # 5 phút

# ===== WINDOW =====
WINDOW_SIZE = 100        # crawl rất sâu → window nhỏ

# ===== ENCODING =====
PAGE_ENCODING = "euc-jp"


In [21]:
PROGRESS_FILE = "crawl_progress.json" # File lưu trạng thái offset và data

In [22]:
def load_checkpoint():
    if os.path.exists(PROGRESS_FILE):
        try:
            with open(PROGRESS_FILE, 'r', encoding='utf-8') as f:
                checkpoint = json.load(f)
                print(f"🔄 Tìm thấy tiến trình cũ: Offset cuối cùng là {checkpoint['last_offset']}")
                return checkpoint['last_offset'], checkpoint['post_list']
        except Exception as e:
            print(f"⚠️ Lỗi đọc file checkpoint, bắt đầu mới: {e}")
    return START_PAGES, []

def save_checkpoint(last_offset, post_list):
    checkpoint = {
        "last_offset": last_offset,
        "post_list": post_list
    }
    with open(PROGRESS_FILE, 'w', encoding='utf-8') as f:
        json.dump(checkpoint, f, ensure_ascii=False, indent=4)
    print(f"💾 Đã lưu checkpoint tại offset {last_offset}. Tổng số bài: {len(post_list)}")

In [23]:
POST_LIST = []

In [24]:

session = requests.Session()
session.headers.update(headers)

In [ ]:
def crawl_offset(ofst: int) -> bool:
    url = BASE_URL + str(ofst)

    response = None

    for attempt in range(MAX_RETRY + 1):
        try:
            response = session.get(
                url,
                timeout=(CONNECT_TIMEOUT, READ_TIMEOUT)
            )
            break
        except requests.exceptions.ReadTimeout:
            wait = BASE_SLEEP * (attempt + 1)
            print(f"[TIMEOUT] ofst={ofst}, wait {wait:.1f}s")
            time.sleep(wait)
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] ofst={ofst}: {e}")
            return False

    if response is None or response.status_code != 200:
        print(f"[SKIP] ofst={ofst}")
        return False

    if len(response.content) < 800:
        print(f"[SKIP] tiny response at ofst={ofst}")
        return False

    response.encoding = PAGE_ENCODING
    soup = BeautifulSoup(response.text, 'html.parser')
    all_trs = soup.find_all('tr')

    current_post = {}

    for tr in all_trs:
        # ===== HEADER =====
        header_td = tr.find('td', attrs={'width': '450', 'colspan': '2'})
        if header_td and not header_td.has_attr('align'):
            fonts = header_td.find_all('font')
            if len(fonts) >= 4:
                current_post = {}
                current_post['user'] = fonts[1].get_text(strip=True)
                current_post['post_name'] = fonts[3].get_text(strip=True)

        # ===== CONTENT =====
        content_td = tr.find(
            'td',
            attrs={'width': '450', 'colspan': '2', 'align': 'left'}
        )
        if content_td and 'user' in current_post:
            fonts = content_td.find_all('font')
            if fonts:
                current_post['post_content'] = fonts[0].get_text(strip=True)
                POST_LIST.append(current_post)
                current_post = {}

    return True


In [26]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from tqdm import tqdm

In [ ]:
from tqdm.auto import tqdm
import time
import random

# Tải tiến trình cũ nếu có
current_start_offset, POST_LIST = load_checkpoint()

# 1. Tính toán tổng số lượng cần crawl để hiển thị thanh tiến trình
total_to_crawl = current_start_offset - LIMIT
consecutive_failures = 0

# 2. Khởi tạo tqdm
with tqdm(total=total_to_crawl, desc="🚀 Đang chuẩn bị...", unit="post") as pbar:
    
    # Duyệt theo từng window
    for window_start in range(current_start_offset, LIMIT, -WINDOW_SIZE):
        window_end = max(window_start - WINDOW_SIZE, LIMIT)
        
        pbar.set_description(f"📂 Window {window_start} → {window_end}")

        for ofst in range(window_start, window_end, -1):
            ok = crawl_offset(ofst)

            if not ok:
                consecutive_failures += 1
                pbar.write(f"⚠️ [FAIL] Offset {ofst} | Liên tiếp: {consecutive_failures}")
            else:
                consecutive_failures = 0

            # Cập nhật thanh tiến trình thêm 1 đơn vị
            pbar.update(1)

            # Xử lý khi bị block
            if consecutive_failures >= FAIL_THRESHOLD:
                pbar.write("\n🚨 [BLOCK DETECTED] - Đang lưu và tạm nghỉ...")
                save_checkpoint(ofst, POST_LIST) 
                
                # Cập nhật trạng thái nghỉ lên thanh tiến trình
                for i in range(BLOCK_PAUSE, 0, -1):
                    pbar.set_description(f"⏸ Nghỉ do Block ({i}s)")
                    time.sleep(1)
                
                consecutive_failures = 0
                break # Thoát vòng lặp offset hiện tại

            # Sleep ngắn giữa các request
            sleep_time = BASE_SLEEP + random.random() * JITTER
            time.sleep(sleep_time)

        # SAU MỖI WINDOW
        save_checkpoint(window_end, POST_LIST)
        pbar.write(f"✅ Đã xong window tới offset {window_end}. Tổng: {len(POST_LIST)}")
        
        # Nghỉ dài giữa các window (trừ khi đã hết dữ liệu)
        if window_end > LIMIT:
            for i in range(BLOCK_PAUSE, 0, -1):
                time.sleep(1)

pbar.write(f"🎉 [DONE] Hoàn tất crawl. Tổng số bài: {len(POST_LIST)}")

🔄 Tìm thấy tiến trình cũ: Offset cuối cùng là 426628


📂 Window 426628 → 426528:   0%|          | 0/728 [00:00<?, ?post/s]

[CRAWLING] ofst=426628


📂 Window 426628 → 426528:   0%|          | 1/728 [00:11<2:20:12, 11.57s/post]

[CRAWLING] ofst=426627


In [ ]:
import json
with open('lily_chou_chou_posts.json', 'w', encoding='utf-8') as f:
    json.dump(POST_LIST, f, ensure_ascii=False, indent=4)